In [3]:
import os

from openai import OpenAI

In [4]:
client = OpenAI()

In [9]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You're a helpful assitant."},
        {"role": "user", "content": "what is a linter in vscode and how it is diff from formatter?"
        }
    ]
)

In [11]:
response = completion.choices[0].message.content
print(response)

A linter and a formatter are both tools used in the development process, often integrated into editors like Visual Studio Code (VSCode), but they serve different purposes.

### Linter
A **linter** is a tool that analyzes your code for potential errors, anti-patterns, and style inconsistencies. It examines your source code and provides feedback to developers about possible programming mistakes, bugs, or stylistic errors. Linters often enforce coding standards and best practices.

- **Purpose**: Identify issues in the code such as syntax errors, type errors, and stylistic problems.
- **Examples**: ESLint for JavaScript, pylint for Python, flake8 for Python, and RuboCop for Ruby.
- **Output**: Warnings and errors; it often highlights issues like unused variables, undefined variables, and code complexity.

### Formatter
A **formatter**, on the other hand, is a tool that automatically formats your code according to specified style guidelines. It adjusts the layout of your code by applying c

In [5]:
from pydantic import BaseModel

class MyEvents(BaseModel):
    name: str
    date: str
    participants: list[str]

In [6]:
completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "Extract the event information"},
        {
            "role": "user", "content": "Suman and Gita are going to a movie on Friday."
        },
    ],
    response_format=MyEvents
)

In [7]:
event = completion.choices[0].message.parsed

In [9]:
print(type(event))

<class '__main__.MyEvents'>


In [10]:
print(event)

name='Movie Night' date='Friday' participants=['Suman', 'Gita']
